In [3]:
import pandas as pd
import numpy as np

from sklearn.multioutput import MultiOutputRegressor

from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error,  r2_score

In [7]:
df=pd.read_csv(r"C:\Users\HP\Downloads\PB_All_2000_2021.csv",sep=';')
df 

,id,date,NH4,BSK5,Suspended,O2,NO3,NO2,SO4,PO4,CL
0,1,17.02.2000,0.330,2.77,12.0,12.30,9.50,0.057,154.00,0.454,289.50
1,1,11.05.2000,0.044,3.00,51.6,14.61,17.75,0.034,352.00,0.090,1792.00
2,1,11.09.2000,0.032,2.10,24.5,9.87,13.80,0.173,416.00,0.200,2509.00
3,1,13.12.2000,0.170,2.23,35.6,12.40,17.13,0.099,275.20,0.377,1264.00
4,1,02.03.2001,0.000,3.03,48.8,14.69,10.00,0.065,281.60,0.134,1462.00
...,...,...,...,...,...,...,...,...,...,...,...
2856,22,06.10.2020,0.046,2.69,3.6,8.28,3.80,0.038,160.00,0.726,77.85
2857,22,27.10.2020,0.000,1.52,0.5,11.26,0.56,0.031,147.20,0.634,71.95
2858,22,03.12.2020,0.034,0.29,0.8,11.09,2.58,0.042,209.92,0.484,61.17
2859,22,12.01.2021,0.000,2.10,0.0,14.31,3.94,0.034,121.60,0.424,63.49


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2861 entries, 0 to 2860
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         2861 non-null   int64  
 1   date       2861 non-null   object 
 2   NH4        2858 non-null   float64
 3   BSK5       2860 non-null   float64
 4   Suspended  2845 non-null   float64
 5   O2         2858 non-null   float64
 6   NO3        2860 non-null   float64
 7   NO2        2858 non-null   float64
 8   SO4        2812 non-null   float64
 9   PO4        2833 non-null   float64
 10  CL         2812 non-null   float64
dtypes: float64(9), int64(1), object(1)
memory usage: 246.0+ KB


In [11]:
df.shape

(2861, 11)

In [13]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,2861.0,12.397064,6.084226,1.00,8.0000,14.000,16.00000,22.000
NH4,2858.0,0.758734,2.486247,0.00,0.0800,0.220,0.50000,39.427
BSK5,2860.0,4.316182,2.973997,0.00,2.1600,3.800,5.80000,50.900
Suspended,2845.0,12.931905,16.543097,0.00,6.0000,10.000,15.00000,595.000
O2,2858.0,9.508902,4.428260,0.00,7.0925,8.995,11.52000,90.000
NO3,2860.0,4.316846,6.881188,0.00,1.3900,2.800,5.58250,133.400
NO2,2858.0,0.246128,2.182777,0.00,0.0300,0.059,0.12575,109.000
SO4,2812.0,59.362313,96.582641,0.00,27.0525,37.800,64.64000,3573.400
PO4,2833.0,0.418626,0.771326,0.00,0.1300,0.270,0.47000,13.879
CL,2812.0,93.731991,394.512184,0.02,26.8000,33.900,45.60750,5615.280


In [15]:
df.isnull().sum()

id            0
date          0
NH4           3
BSK5          1
Suspended    16
O2            3
NO3           1
NO2           3
SO4          49
PO4          28
CL           49
dtype: int64

In [19]:
df['date']=pd.to_datetime(df['date'],format='%d.%m.%Y')
df

,id,date,NH4,BSK5,Suspended,O2,NO3,NO2,SO4,PO4,CL
0,1,2000-02-17,0.330,2.77,12.0,12.30,9.50,0.057,154.00,0.454,289.50
1,1,2000-05-11,0.044,3.00,51.6,14.61,17.75,0.034,352.00,0.090,1792.00
2,1,2000-09-11,0.032,2.10,24.5,9.87,13.80,0.173,416.00,0.200,2509.00
3,1,2000-12-13,0.170,2.23,35.6,12.40,17.13,0.099,275.20,0.377,1264.00
4,1,2001-03-02,0.000,3.03,48.8,14.69,10.00,0.065,281.60,0.134,1462.00
...,...,...,...,...,...,...,...,...,...,...,...
2856,22,2020-10-06,0.046,2.69,3.6,8.28,3.80,0.038,160.00,0.726,77.85
2857,22,2020-10-27,0.000,1.52,0.5,11.26,0.56,0.031,147.20,0.634,71.95
2858,22,2020-12-03,0.034,0.29,0.8,11.09,2.58,0.042,209.92,0.484,61.17
2859,22,2021-01-12,0.000,2.10,0.0,14.31,3.94,0.034,121.60,0.424,63.49


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2861 entries, 0 to 2860
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   id         2861 non-null   int64         
 1   date       2861 non-null   datetime64[ns]
 2   NH4        2858 non-null   float64       
 3   BSK5       2860 non-null   float64       
 4   Suspended  2845 non-null   float64       
 5   O2         2858 non-null   float64       
 6   NO3        2860 non-null   float64       
 7   NO2        2858 non-null   float64       
 8   SO4        2812 non-null   float64       
 9   PO4        2833 non-null   float64       
 10  CL         2812 non-null   float64       
dtypes: datetime64[ns](1), float64(9), int64(1)
memory usage: 246.0 KB


In [23]:
df=df.sort_values(by=['id','date'])
df.head()

,id,date,NH4,BSK5,Suspended,O2,NO3,NO2,SO4,PO4,CL
0,1,2000-02-17,0.330,2.77,12.0,12.30,9.50,0.057,154.0,0.454,289.5
1,1,2000-05-11,0.044,3.00,51.6,14.61,17.75,0.034,352.0,0.090,1792.0
2,1,2000-09-11,0.032,2.10,24.5,9.87,13.80,0.173,416.0,0.200,2509.0
3,1,2000-12-13,0.170,2.23,35.6,12.40,17.13,0.099,275.2,0.377,1264.0
4,1,2001-03-02,0.000,3.03,48.8,14.69,10.00,0.065,281.6,0.134,1462.0


In [25]:
df['year']=df['date'].dt.year
df['month']=df['date'].dt.month

In [27]:
df.head()

,id,date,NH4,BSK5,Suspended,O2,NO3,NO2,SO4,PO4,CL,year,month
0,1,2000-02-17,0.330,2.77,12.0,12.30,9.50,0.057,154.0,0.454,289.5,2000,2
1,1,2000-05-11,0.044,3.00,51.6,14.61,17.75,0.034,352.0,0.090,1792.0,2000,5
2,1,2000-09-11,0.032,2.10,24.5,9.87,13.80,0.173,416.0,0.200,2509.0,2000,9
3,1,2000-12-13,0.170,2.23,35.6,12.40,17.13,0.099,275.2,0.377,1264.0,2000,12
4,1,2001-03-02,0.000,3.03,48.8,14.69,10.00,0.065,281.6,0.134,1462.0,2001,3


In [31]:
df.columns

Index(['id', 'date', 'NH4', 'BSK5', 'Suspended', 'O2', 'NO3', 'NO2', 'SO4',
       'PO4', 'CL', 'year', 'month'],
      dtype='object')

In [53]:
# Cell 1: Define pollutants (if not already defined in the current session)
pollutants = ['O2', 'NO3', 'NO2', 'SO4', 'PO4', 'CL'] # Ensure 'NO2' is included if you want it

# Cell 2: Verify DataFrame columns and pollutants list right before dropna
print("Columns in df:", df.columns.tolist())
print("Pollutants list:", pollutants)

# Check if all pollutants are actually in df.columns
missing_cols = [col for col in pollutants if col not in df.columns]
if missing_cols:
    print(f"Error: The following pollutant columns are NOT in the DataFrame: {missing_cols}")
else:
    print("All pollutant columns are present in the DataFrame.")

# Now, attempt the dropna()
df_cleaned = df.dropna(subset=pollutants) # Assign to a new variable to be safe
print("Successfully ran dropna. Shape after dropna:", df_cleaned.shape)
print("Missing values after dropna:", df_cleaned.isnull().sum())
df_cleaned.head()

Columns in df: ['id', 'date', 'NH4', 'BSK5', 'Suspended', 'O2', 'NO3', 'NO2', 'SO4', 'PO4', 'CL', 'year', 'month']
Pollutants list: ['O2', 'NO3', 'NO2', 'SO4', 'PO4', 'CL']
All pollutant columns are present in the DataFrame.
Successfully ran dropna. Shape after dropna: (2780, 13)
Missing values after dropna: id           0
date         0
NH4          2
BSK5         0
Suspended    2
O2           0
NO3          0
NO2          0
SO4          0
PO4          0
CL           0
year         0
month        0
dtype: int64


,id,date,NH4,BSK5,Suspended,O2,NO3,NO2,SO4,PO4,CL,year,month
0,1,2000-02-17,0.330,2.77,12.0,12.30,9.50,0.057,154.0,0.454,289.5,2000,2
1,1,2000-05-11,0.044,3.00,51.6,14.61,17.75,0.034,352.0,0.090,1792.0,2000,5
2,1,2000-09-11,0.032,2.10,24.5,9.87,13.80,0.173,416.0,0.200,2509.0,2000,9
3,1,2000-12-13,0.170,2.23,35.6,12.40,17.13,0.099,275.2,0.377,1264.0,2000,12
4,1,2001-03-02,0.000,3.03,48.8,14.69,10.00,0.065,281.6,0.134,1462.0,2001,3


In [55]:
x=df[['id','year']]
y=df[pollutants]

In [57]:
x_encoded=pd.get_dummies(x,columns=['id'],drop_first=True)

In [59]:
x_train,x_test,y_train,y_test=train_test_split(x_encoded,y,test_size=0.2,random_state=42)

In [86]:
import numpy as np
import pandas as pd # Assuming you might be using pandas DataFrames
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# --- Dummy Data for Demonstration ---
# Replace this with your actual loading and preprocessing of X_train and y_train
# Ensure X_train has features and y_train has targets.

# Example 1: Single output, needs reshaping
# X_train = np.random.rand(100, 10) # 100 samples, 10 features
# y_train = np.random.rand(100)    # 100 samples, 1D target

# Example 2: Multi-output already 2D
X_train = np.random.rand(100, 10) # 100 samples, 10 features
y_train = np.random.rand(100, 2)  # 100 samples, 2 target outputs

# Example 3: If X_train or y_train are pandas DataFrames/Series
# X_train = pd.DataFrame(np.random.rand(100, 10), columns=[f'feature_{i}' for i in range(10)])
# y_train = pd.Series(np.random.rand(100)) # Needs reshaping if a Series
# y_train = pd.DataFrame(np.random.rand(100, 2), columns=['target_1', 'target_2']) # Already 2D if a DataFrame with multiple columns


# --- IMPORTANT DEBUGGING STEPS ---
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Number of dimensions in y_train: {y_train.ndim}")
print(f"Type of X_train: {type(X_train)}")
print(f"Type of y_train: {type(y_train)}")

# Check for NaN values (if using pandas, otherwise you can convert to numpy and check)
if isinstance(X_train, (pd.DataFrame, pd.Series)):
    print(f"NaNs in X_train: {X_train.isnull().sum().sum()}")
else: # Assuming numpy array
    print(f"NaNs in X_train: {np.isnan(X_train).sum()}")

if isinstance(y_train, (pd.DataFrame, pd.Series)):
    print(f"NaNs in y_train: {y_train.isnull().sum().sum()}")
else: # Assuming numpy array
    print(f"NaNs in y_train: {np.isnan(y_train).sum()}")


# --- THE POTENTIAL FIXES (apply based on your data) ---

# 1. Ensure y_train is 2D:
# If y_train is a pandas Series or a 1D numpy array, reshape it.
if y_train.ndim == 1:
    print("Reshaping y_train to 2D (n_samples, 1)")
    y_train = y_train.values.reshape(-1, 1) if isinstance(y_train, pd.Series) else y_train.reshape(-1, 1)

# 2. Convert to NumPy arrays if they are pandas DataFrames/Series
# MultiOutputRegressor and RandomForestRegressor typically work well with pandas,
# but converting explicitly to NumPy can sometimes resolve subtle issues.
if isinstance(X_train, (pd.DataFrame, pd.Series)):
    print("Converting X_train to NumPy array.")
    X_train = X_train.values
if isinstance(y_train, (pd.DataFrame, pd.Series)):
    print("Converting y_train to NumPy array.")
    y_train = y_train.values

# 3. Handle NaN values (if any were printed above)
# You would typically do this much earlier in your preprocessing.
# Example: Impute missing values with the mean (or median, or drop rows)
# from sklearn.impute import SimpleImputer
# imputer = SimpleImputer(strategy='mean')
# X_train = imputer.fit_transform(X_train)
# y_train = imputer.fit_transform(y_train) # Only if y can have NaNs and you want to impute


print(f"\n--- After potential fixes ---")
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Number of dimensions in y_train: {y_train.ndim}")


# --- Your original model code ---
model = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42))
model.fit(X_train, y_train)

print("\nModel fitted successfully!")

Shape of X_train: (100, 10)
Shape of y_train: (100, 2)
Number of dimensions in y_train: 2
Type of X_train: <class 'numpy.ndarray'>
Type of y_train: <class 'numpy.ndarray'>
NaNs in X_train: 0
NaNs in y_train: 0

--- After potential fixes ---
Shape of X_train: (100, 10)
Shape of y_train: (100, 2)
Number of dimensions in y_train: 2

Model fitted successfully!


In [90]:
import numpy as np
import pandas as pd # Assuming you might be using pandas DataFrames
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# --- 1. Generate/Load your Data (Crucial Step - Ensure consistency) ---
# It's vital that X_train, X_test, y_train are correctly prepared.
# For demonstration, let's create consistent dummy data:
# Assume you have 100 samples, 10 features, and 2 output targets.

# Create dummy feature names for demonstration of the UserWarning fix
feature_names = [f'feature_{i}' for i in range(10)]

X = pd.DataFrame(np.random.rand(120, 10), columns=feature_names)
y = pd.DataFrame(np.random.rand(120, 2), columns=['target_1', 'target_2'])

# Split the data into training and testing sets
# Ensure random_state for reproducibility
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- 2. Data Preprocessing & Verification (as discussed before) ---
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of X_test: {X_test.shape}") # Check X_test shape too
print(f"Number of dimensions in y_train: {y_train.ndim}")

# Ensure y_train is 2D (for MultiOutputRegressor)
# This handles both pandas Series and 1D numpy arrays
if y_train.ndim == 1:
    print("Reshaping y_train to 2D (n_samples, 1).")
    # Use .values if it's a pandas Series to get the underlying numpy array
    y_train = y_train.values.reshape(-1, 1)
elif isinstance(y_train, pd.Series): # If it's a Series but already has multiple values, it will be 1D, so reshape
    print("Converting pandas Series y_train to 2D numpy array.")
    y_train = y_train.values.reshape(-1, 1)


# --- 3. Model Initialization and Fitting ---
# The MultiOutputRegressor creates an instance of RandomForestRegressor for each output.
# The previous fit error was resolved by ensuring y_train is 2D.
model = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42))

try:
    model.fit(X_train, y_train)
    print("\nModel fitted successfully!")
except ValueError as e:
    print(f"\nError during model fitting: {e}")
    # Add more debugging if fitting still fails, like checking for NaNs in X_train/y_train.
    # print(X_train.isnull().sum().sum())
    # print(y_train.isnull().sum().sum())
    exit() # Stop execution if fit fails

# --- 4. Prediction (Addressing the new error and UserWarning) ---

# To avoid the 'X has feature names' UserWarning and potential column mismatch issues:
# Ensure X_test has the same column names and order as X_train.
# If X_train and X_test are DataFrames:
if isinstance(X_train, pd.DataFrame) and isinstance(X_test, pd.DataFrame):
    # This ensures X_test columns match X_train columns, filling missing with NaN if necessary
    # and dropping extra columns in X_test.
    X_test_aligned = X_test[X_train.columns]
    print("\nAligned X_test columns for prediction.")
else: # If they are numpy arrays, this alignment isn't necessary, but ensure consistent order
    X_test_aligned = X_test

try:
    y_pred = model.predict(X_test_aligned)
    print(f"Shape of y_pred: {y_pred.shape}")
    print("\nPrediction successful!")
except ValueError as e:
    print(f"\nError during prediction: {e}")
    print("This specific 'base estimator should have a predict method' error, after successful fit,")
    print("is very rare for RandomForestRegressor. It might indicate a subtle issue with the")
    print("internal state of the fitted base estimators or potentially a version-specific bug.")
    print("Double-check your sklearn version and consider updating it if it's old.")
    print("Ensure no NaN values are introduced after fitting and before prediction in X_test_aligned.")
    # You might also try:
    # for i, estimator in enumerate(model.estimators_):
    #     print(f"Estimator {i} has predict method: {'predict' in dir(estimator)}")

Shape of X_train: (96, 10)
Shape of y_train: (96, 2)
Shape of X_test: (24, 10)
Number of dimensions in y_train: 2

Model fitted successfully!

Aligned X_test columns for prediction.
Shape of y_pred: (24, 2)

Prediction successful!


In [7]:


print("Model Performance on the Test Data:")


for i, pollutant in enumerate (pollutants):

   print(f'{pollutant}:')

   print(' MSE:', mean_squared_error(y_test.iloc[:, i], y_pred[:, i]))

   print(' R2:', r2_score(y_test.iloc[:, i], y_pred[:, i]))

   print()

Model Performance on the Test Data:


NameError: name 'pollutants' is not defined